In [1]:
import requests
import pandas as pd

In [52]:
url_uf = "https://resultados.tse.jus.br/oficial/ele2022/544/config/mun-e000544-cm.json"
requisicao_uf = requests.get(url_uf)
requisicao_uf = requisicao_uf.json()

In [ ]:
lista_candidatos = [] 

for uf in requisicao_uf["abr"]:
    
    print(uf["ds"])

    sigla_uf = uf["cd"]
    estado = uf["ds"]

    print("------")
    for municipio in uf["mu"]:

        nome_mun = municipio["nm"]
        cod_tse = municipio["cd"]
        cod_ibge = municipio["cdi"]

        url_mun = f"https://resultados.tse.jus.br/oficial/ele2022/544/dados/{sigla_uf.lower()}/{sigla_uf.lower()}{cod_tse}-c0001-e000544-v.json"
        requisicao_mun = requests.get(url_mun)
        requisicao_mun = requisicao_mun.json()

        for candidato in requisicao_mun["abr"][0]["cand"]:

            lista_candidatos.append({
                "uf": sigla_uf,
                "estado": estado,
                "nome_mun": nome_mun,
                "cod_tse": cod_tse,
                "cod_ibge": cod_ibge,
                "posicao_canditato": candidato["seq"],
                "num_canditato": candidato["n"],
                "situacao": candidato["st"],
                "votos_totais": candidato["vap"],
                "perc_votos_totais": candidato["pvap"],
            })


In [61]:
print(len(lista_candidatos))

63261


In [62]:
df_dados = pd.DataFrame(lista_candidatos)
df_dados.to_csv("votos_presidente_municipios_2022.csv")